# Projeto 1 - Ciência dos Dados

Nome: ROBERTO SCHNORRENBERG

Nome: RAPHAEL BUTORI

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Obtenção dos tweets

## Preparando o ambiente no jupyter:

In [2]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [1]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import numpy as np

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [4]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas para construção da base de dados:

### Escolha de um produto e coleta das mensagens


In [5]:
#Produto escolhido:
produto = 'spotify'

#Quantidade mínima de mensagens capturadas:
n = 2000
#Quantidade mínima de mensagens para a base de treinamento:
t= 700

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [6]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

TweepError: Twitter error response: status code = 429

In [7]:
msgs=list(set(list(msgs)))

Salvando os dados em uma planilha Excel:

In [86]:
import string
#string punctuation contem todas as pontuacoes ja
pontuacao = string.punctuation
ponto = ["\n", "\t"]

#criando uma lista
limpeza = []
for tweet in msgs:
    x = ''
    for letra in tweet:
        if tweet[0] != 'r' or tweet[1]!= 't':
            if letra in ponto:
                x += " "
            elif letra not in pontuacao:
                x += letra
    if x!= '':
        limpeza.append(x)


NameError: name 'msgs' is not defined

In [13]:
print(len(limpeza))

629


___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

**Importante: Caso classifique um percentual pequeno de tweets relevantes ou de não relevantes, deve voltar a este notebook e coletar mais tweets diferentes do produto escolhido pelo grupo.**

In [190]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
msgs=limpeza
if not os.path.isfile('./{0}classificado.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}classificado.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:300])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[300:400])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

NameError: name 'produto' is not defined

In [16]:
sptf=pd.read_excel('spotify_novo_class.xlsx')
sptf2=pd.read_excel('spotifyclassificado.xlsx')
dfs=[sptf,sptf2]
spt=pd.concat(dfs)
spt.head()

C:\Users\rbuto\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


,RELEVANCIA,Treinamento,Unnamed: 1,legenda
0,1.0,eu já chorei d com essa aqui httpstcozppqt7nheu,NaN,NaN
1,2.0,spotify eu te amo sempre com as melhores ind...,NaN,NaN
2,1.0,siceramenteeee bom demais o que que pega com f...,NaN,NaN
3,1.0,depressao httpstcoh7su37ctwd,NaN,NaN
4,4.0,hoseok eu consigo imaginar você cantando essa ...,NaN,NaN


In [17]:
#vocabulario total (sem repetição)

palavrastot=[]
for frase in spt.Treinamento:
    tweet=frase.split()
    for palavra in tweet:
        if palavra not in palavrastot:
            palavrastot.append(palavra)

In [18]:
print(len(palavrastot))

3236


In [19]:
#separacao em categprias
irrelevante=spt.loc[spt.RELEVANCIA==1]
marca=spt.loc[spt.RELEVANCIA==2]
artista_album_genero=spt.loc[spt.RELEVANCIA==3]
musica=spt.loc[spt.RELEVANCIA==4]

In [20]:
#apenas tweets de cada categoria
i=irrelevante.Treinamento
m1=marca.Treinamento
a=artista_album_genero.Treinamento
m2=musica.Treinamento

In [21]:
#palavras na categoria (com repeticao)
def palcat(categoria):
    lpal=[]
    for frase in categoria:
        tweet=frase.split()
        for palavra in tweet:
            lpal.append(palavra)
    return lpal
#ok


In [22]:
palcati=palcat(i)
palcatm1=palcat(m1)
palcata=palcat(a)
palcatm2=palcat(m2)


In [23]:
len(palcati)

1449

# Smoothing de Laplace

Nesta etapa garante-se de que nenhum resultado será equivalente a 0 atribuindo um valor muito pequeno de relevância às palavras não mencionadas na categoria em questão

In [24]:
#frequencia relativa das palavras aplicando smoothing de laplace
def laplaceando(palcat,palavrastot):
    freq_abs=pd.Series(palcat)
    fr={}
    ja_contadas=[]
    for x in palcat:
        if x not in ja_contadas:
            count=freq_abs.value_counts(normalize=False)
            b=(count[x]+1)/(len(palcat)+len(palavrastot))
            fr[x]=b
    return fr

lpci=laplaceando(palcati,palavrastot)
lpcm1=laplaceando(palcatm1,palavrastot)
lpca=laplaceando(palcata,palavrastot)
lpcm2=laplaceando(palcatm2,palavrastot)

In [25]:
#denominador a ser utilizado para cada categoria(total de palavras+ frequencia absolutas na categoria)

# Função classificadora

Nessa função damos como argumentos as variáveis utilizadas préviamente calculadas com a única alteração necessária sendo a da frase em questão, que posteriormente deve ser substituída pela qual queira classificar

In [26]:
def classificador(lpci,lpcm1,lpca,lpcm2,frase,palavrastot,palcati,palcatm1,palcata,palcatm2):
    
    valorpalfrasei=[]
    valorpalfrasem1=[]
    valorpalfrasea=[]
    valorpalfrasem2=[]
    for x in frase:
        
        if x not in lpci:
            lpci[x]= 1/(len(palavrastot)+len(palcati))
        valorpalfrasei.append(lpci[x])
            
        if x not in lpcm1:
            lpcm1[x]= 1/(len(palavrastot)+len(palcatm1))
        valorpalfrasem1.append(lpcm1[x])
        
        if x not in lpca:
            lpca[x]= 1/(len(palavrastot)+len(palcata))
        valorpalfrasea.append(lpca[x])
        
        if x not in lpcm2:
            lpcm2[x]= 1/(len(palavrastot)+len(palcatm2))
        valorpalfrasem2.append(lpcm2[x])

    vfi=1
    vfm1=1
    vfa=1
    vfm2=1
    
    for x in valorpalfrasei:
        vfi*=x
        
    for x in valorpalfrasem1:
        vfm1*=x

    for x in valorpalfrasea:
        vfa*=x
        
    for x in valorpalfrasem2:
        vfm2*=x
    
    velhotruquedomaior=[0,vfi,vfm1,vfa,vfm2]
    
    x=0
    maior=0
    vmaior=0
    while x<5:
        if velhotruquedomaior[x]>vmaior:
            maior=x
            vmaior=velhotruquedomaior[x]
        x+=1
    
    classe=0
    if maior==1:
        classe='irrelevante'
    elif maior==2:
        classe='marca'
    elif maior==3:
        classe='artista/genero/album'
    elif maior==4:
        classe='musica'
        
    return maior

In [27]:
frase='musica da boa'
TESTE=classificador(lpci,lpcm1,lpca,lpcm2,frase,palavrastot,palcati,palcatm1,palcata,palcatm2)

In [28]:
print(TESTE)

1


In [30]:
teste1=pd.read_excel( 'spotifyclassificado.xlsx' , 'Teste')
teste2=pd.read_excel('spotify_novo_class.xlsx','Teste')
dfs2=[teste1,teste2]
teste=pd.concat(dfs2,sort=True)

teste["classificado"]=0
teste.set_index('classificado')

teste.head()


,RELEVANCIA,Teste,Unnamed: 1,classificado
0,NaN,pq o app do spotify pra smartv é tão ruim,2.0,0
1,NaN,httpstcoqsbs4uczot me chame de tiozão mas iss...,1.0,0
2,NaN,é que a distância aumenta a vontade de te ter ...,1.0,0
3,NaN,gente vcs sabem se da pra mudar o nome de usuá...,2.0,0
4,NaN,umusicmexico raymixmusic paulinarubio spotifym...,1.0,0


In [39]:
for x in range( len(teste.Teste)):
    teste.classificado.iloc[x]=classificador(lpci,lpcm1,lpca,lpcm2,teste.Teste.iloc[x],palavrastot,palcati,palcatm1,palcata,palcatm2)

C:\Users\rbuto\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [40]:
teste.head(39)

,RELEVANCIA,Teste,Unnamed: 1,classificado
0,NaN,pq o app do spotify pra smartv é tão ruim,2.0,3
1,NaN,httpstcoqsbs4uczot me chame de tiozão mas iss...,1.0,3
2,NaN,é que a distância aumenta a vontade de te ter ...,1.0,1
3,NaN,gente vcs sabem se da pra mudar o nome de usuá...,2.0,1
4,NaN,umusicmexico raymixmusic paulinarubio spotifym...,1.0,0
5,NaN,a melhor da lauren e so minha opinião importa ...,1.0,1
6,NaN,aqui está uma música para você… ribs lorde htt...,4.0,3
7,NaN,zirve rota 👊 httpstcor2vkdgdh6a,1.0,3
8,NaN,o começo me lembra motivation httpstcokc0h2capzw,1.0,3
9,NaN,tupisports spotify paudecast não se briga com ...,1.0,1


In [32]:
a=classificador(lpci,lpcm1,lpca,lpcm2,teste.Teste[18],palavrastot,palcati,palcatm1,palcata,palcatm2)
print(a)

2


## CONCLUSÃO

   Tal projeto ilustra o Classificador de Naive Bayes, com a atribuição de percentuais aos elementos analisados em questão referentes ao total de possibilidades, essencialmente é feita uma probabilidade composta atribuida aos valores que cada palavra representa para categoria em questão. A escolha da marca foi realizada afim de vizualizar a frequência com a qual cad aum dos tópicos é associado com a marca, em termos de sua utilidade para a marca as mensagens uma vez classificadas poderiam ser subclassificadas por exemplo em positivas e negativas em uma etapa posterior uma vez que já nas mãos da subdivisão da empresa a qual os dados interessam, assim podendo evidenciar quais as melhorias possíveis em certa área sem que haja um grande percentual de informações irrelevantes para o setor direcionada a ele. A separação foi então feita nos seguintes grupos:

* 1-Menções irrelevantes: com intuito de separar as mensagens que não seram úteis aos donos da marca.
* 2-Menções em relação a marca Spotify: com intuito de apresentar as opiniões e críticas sobre a marca em si.
* 3-Menções sobre artistas, álbuns ou genêro: com intuito de apresentar por exemplo artistas, genêros ou álbuns que ainda não estão disponiveis no aplicativo e que os usuários sentem falta.
* 4-Menções sobre uma música específica: com intuito de apresentar músicas que também faltam no aplicativo.

    Além da subdivisão posterior mencionada anteriormente, também vemos como possíveis áreas com espaço para aprimoramento a abordagem à possíveis mensagens que abordam dois assuntos no mesmo tweet , como por exemplo mencionar um artista e sua música em uma mesma postagem, como abordagem a esse problema faríamos a sugestão de duplicar o tweet e classifica-lo duas vezes uma com cada classificação, ou a adoção de uma segunda coluna de classificação secundária caso os índices de ocorrência do mesmo atingissem números que demonstrem necessidade do mesmo, tanto a primeira como segunda abordagem poderiam ser executadas com adoção de uma porcentagem "gate" para o cenário.
    ### por que não alimentar o treinamento por meio do classificador
    a abordagem em questão evita a ampliação do dicionário bem como não trás grandes benefícios para o classificador , uma base gerada pelo classificador teria frases sem nexo , sem palavras novas e resultariam ao fim em um resultado aleatório para cada palavra podendo a precisão aumentar ou diminuir com relação a cada palavra de acordo com o contexto na qual ela foi inserida(frase, também aleatoriamente)

**Outra Aplicação**

Uma outra aplicação para o Bayes seria por exemplo o processamento de linguagem natural e diagnósticos médicos, para contextos com instâncias condicionalmente independentes. Ou seja, o Teorema de Bayes com sua probabilidade condicional de um evento A ocorrer, dado o evento B. Portanto, com seus devidos dados é possível calcular a probabilidade de algum paciente não possuir uma doença qualquer, dado que ele recebeu um resultado positivo em relação a possuir a doença, podendo contar com a associação de características como hábito de fumar e a relação que isso teria com câncer.

